Revised, modified and adapted by Giacomo Gonella.
Revised by Paolo Bresolin and Pietro Picardi.

**Connect google drive**

In [ ]:
#Check if the NVIDIA GPU is enabled
!nvidia-smi

Wed Jul 27 08:59:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Mount the google drive content, in order to access the dataset and store the weights
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 C++		   'Esame di stato 2018'  'My Drive'  'Tesi triennale'
'Colab Notebooks'  'Libri di studio'	  'QR codes'   yolov3


**1) Clone the Darknet**



In [ ]:
#Clone the darknet from an existing model
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
#Change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
     

**3) Configure Darknet network for training YOLO V3**

In [ ]:
#Configure the network
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
#Create a folder in google drive so that we can save there trained the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Hand" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
#Download the weights of the darknet model
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-07-27 09:00:23--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.3’

darknet53.conv.74.3 100%[===================>] 154.96M  52.3MB/s    in 3.0s    

2022-07-27 09:00:27 (52.3 MB/s) - ‘darknet53.conv.74.3’ saved [162482580/162482580]



**4) Extract Images**

In [ ]:
#Unzip the training set
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
replace data/obj/10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/obj/10.jpg         
  inflating: data/obj/10.txt         
  inflating: data/obj/100.jpg        
 extracting: data/obj/100.txt        
  inflating: data/obj/1000.jpg       
  inflating: data/obj/1000.txt       
  inflating: data/obj/1001.jpg       
  inflating: data/obj/1001.txt       
 extracting: data/obj/1002.jpg       
  inflating: data/obj/1002.txt       
 extracting: data/obj/1003.jpg       
  inflating: data/obj/1003.txt       
 extracting: data/obj/1004.jpg       
  inflating: data/obj/1004.txt       
  inflating: data/obj/1005.jpg       
  inflating: data/obj/1005.txt       
  inflating: data/obj/1006.jpg       
  inflating: data/obj/1006.txt       
 extracting: data/obj/1007.jpg       
  inflating: data/obj/1007.txt       
  inflating: data/obj/1008.jpg       
  inflating: data/obj/1008.txt       
  inflating: data/obj/1009.jpg       
  inflating: data/obj/10

In [ ]:
#Access images and print them
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/279.jpg', 'data/obj/1018.jpg', 'data/obj/CHESS_LIVINGROOM_B_S_frame_0051.jpg', 'data/obj/110.jpg', 'data/obj/CHESS_COURTYARD_B_T_frame_1475.jpg', 'data/obj/1029.jpg', 'data/obj/JENGA_OFFICE_S_B_frame_0280.jpg', 'data/obj/1052.jpg', 'data/obj/1040.jpg', 'data/obj/JENGA_OFFICE_T_H_frame_1500.jpg', 'data/obj/JENGA_LIVINGROOM_H_B_frame_0129.jpg', 'data/obj/PUZZLE_COURTYARD_B_S_frame_1496.jpg', 'data/obj/CHESS_OFFICE_B_S_frame_0403.jpg', 'data/obj/CARDS_OFFICE_S_B_frame_0969.jpg', 'data/obj/CARDS_OFFICE_H_T_frame_1316.jpg', 'data/obj/271.jpg', 'data/obj/CHESS_OFFICE_S_B_frame_1103.jpg', 'data/obj/1088.jpg', 'data/obj/CHESS_LIVINGROOM_B_S_frame_2387.jpg', 'data/obj/1048.jpg', 'data/obj/266.jpg', 'data/obj/CHESS_COURTYARD_B_T_frame_0446.jpg', 'data/obj/CHESS_COURTYARD_B_T_frame_0108.jpg', 'data/obj/JENGA_COURTYARD_B_H_frame_0212.jpg', 'data/obj/229.jpg', 'data/obj/PUZZLE_COURTYARD_B_S_frame_0106.jpg', 'data/obj/CHESS_COURTYARD_T_B_frame_1753.jpg', 'data/obj/JENGA_COURTYARD_B_H_fram

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
#Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Sho